In [28]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
video_id = "Gfr50f6ZBvo"  

try:
    # If you don't care which language, this returns the "best" one
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

    # Flatten it to plain text
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)

except Exception as e:
    if "No transcripts were found" in str(e):
        print("No captions available for this video.")
    else:
        print(f"An error occurred: {e}")

An error occurred: no element found: line 1, column 0


In [30]:
Splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = Splitter.create_documents([transcript])
chunks

NameError: name 'transcript' is not defined

In [ ]:
len(chunks)

168

In [ ]:
embeddings = OpenAIEmbeddings(model = "text-embedding-3-small")
vector_store = FAISS.from_documents(chunks, embeddings)

In [ ]:
vector_store.index_to_docstore_id

{0: 'd5fee8f5-8d8c-44dd-b863-7eedb54de945',
 1: 'd2ceeac4-2448-47eb-884f-14d8b32090c3',
 2: 'c9bc7723-f913-4ac1-813b-dfc51c15923a',
 3: '8a831921-2286-48ca-9df0-d1fea8fdfa47',
 4: 'f32899ff-87c8-44ff-8c36-4f8240b6c2c3',
 5: 'c29b785b-6301-4ae2-ac80-bdf2a8433f40',
 6: '04fcc137-0649-452e-af6f-a0ab83a2e0bc',
 7: 'a03f87e9-e9ec-455e-b6f5-4a0b94f701e0',
 8: '8bb4137e-9624-4193-adfe-9a83fb30be68',
 9: 'f779deaf-c72f-4bb4-9d14-668d471aec73',
 10: 'd61d9e1e-2dee-4be9-8998-c48446352768',
 11: '9c36f053-d30a-46e0-88c1-d8f2570b439f',
 12: 'af96f5cc-fa1e-4c0d-bffe-d0726a3aab3f',
 13: '7f932be9-cb0f-48c9-82c4-d2764d6454b4',
 14: 'b5d1240c-aad5-4023-8874-b5fc99c8367d',
 15: '21dec523-739e-4ab0-889b-9e89c9f323df',
 16: '85aaa438-67c6-4642-973e-31db873084f2',
 17: 'd572d0f3-53f7-4d8f-a3b0-91c063ac99c6',
 18: '2f11f1f3-7cea-451e-aff0-bcdd2e100a8e',
 19: '1080bdeb-800d-4886-9806-fbb9161fe47c',
 20: '9dfd5b13-f3d5-439f-b301-1bb27f4951b3',
 21: 'e99f7562-4fe2-42a5-8003-5ffa30aa780b',
 22: '32436703-8b17-

In [ ]:
retriever = vector_store.as_retriever(search_type = "similarity", search_kwargs = {"k": 4}) 

In [ ]:
retriever.invoke("What is deepmind")

[Document(id='d5fee8f5-8d8c-44dd-b863-7eedb54de945', metadata={}, page_content="the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal qu

In [ ]:
llm = ChatOpenAI(model= "gpt-4o-mini",temperature=0.2)

In [ ]:
prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\n      You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      {context}\n      Question: {question}\n    ")

In [ ]:
question          = "Can you summarize the video"
retrieved_docs    = retriever.invoke(question)
retrieved_docs

[Document(id='40c548f6-9283-4d0d-ad9d-bcef83a89bbc', metadata={}, page_content="deeper maybe simpler explanation yes of things right than the standard model of physics which we know doesn't work but we still keep adding to so um and and that's how i think the beginning of an explanation would look and it would start encompassing many of the mysteries that we have wondered about for thousands of years like you know consciousness uh life and gravity all of these things yeah giving us a glimpses of explanations for those things yeah well um damas dear one of the special human beings in this giant puzzle of ours and it's a huge honor that you would take a pause from the bigger puzzle to solve this small puzzle of a conversation with me today it's truly an honor and a pleasure thank you thank you i really enjoyed it thanks lex thanks for listening to this conversation with demas establish to support this podcast please check out our sponsors in the description and now let me leave you with 

In [ ]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"deeper maybe simpler explanation yes of things right than the standard model of physics which we know doesn't work but we still keep adding to so um and and that's how i think the beginning of an explanation would look and it would start encompassing many of the mysteries that we have wondered about for thousands of years like you know consciousness uh life and gravity all of these things yeah giving us a glimpses of explanations for those things yeah well um damas dear one of the special human beings in this giant puzzle of ours and it's a huge honor that you would take a pause from the bigger puzzle to solve this small puzzle of a conversation with me today it's truly an honor and a pleasure thank you thank you i really enjoyed it thanks lex thanks for listening to this conversation with demas establish to support this podcast please check out our sponsors in the description and now let me leave you with some words from edskar dykstra computer science is no more about computers than

In [ ]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [ ]:
answer = llm.invoke(final_prompt)
print(answer.content)

The video features a conversation with Demas, discussing the need for deeper and simpler explanations in physics beyond the standard model, which is known to be inadequate. They touch on various mysteries such as consciousness, life, and gravity, suggesting that a more fundamental understanding could provide insights into these topics. Demas also shares insights about advancements in fusion energy, specifically how they have developed a method to hold plasma in specific shapes for extended periods, which is a significant step in fusion research. Additionally, they mention work on modeling the quantum mechanical behavior of electrons and the potential role of AI in this area. The conversation emphasizes the importance of exploring new frontiers in both physics and technology.


---

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:

video_id = "Gfr50f6ZBvo"  

try:
    # If you don't care which language, this returns the "best" one
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id, languages=["en"])

    # Flatten it to plain text
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)

except Exception as e:
    if "No transcripts were found" in str(e):
        print("No captions available for this video.")
    else:
        print(f"An error occurred: {e}")

An error occurred: no element found: line 1, column 0


In [ ]:

prompt = PromptTemplate(
    template="""
      You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables = ['context', 'question']
)
prompt

In [ ]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [ ]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [ ]:
result = parallel_chain.invoke("What is deepmind")
print(result['context'])

the following is a conversation with demus hasabis ceo and co-founder of deepmind a company that has published and builds some of the most incredible artificial intelligence systems in the history of computing including alfred zero that learned all by itself to play the game of gold better than any human in the world and alpha fold two that solved protein folding both tasks considered nearly impossible for a very long time demus is widely considered to be one of the most brilliant and impactful humans in the history of artificial intelligence and science and engineering in general this was truly an honor and a pleasure for me to finally sit down with him for this conversation and i'm sure we will talk many times again in the future this is the lex friedman podcast to support it please check out our sponsors in the description and now dear friends here's demis hassabis let's start with a bit of a personal question am i an ai program you wrote to interview people until i get good enough


In [ ]:
parser = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:

main_chain.invoke('Can you summarize the video')

'The video features a conversation with Demas, who discusses the limitations of the standard model of physics and the need for deeper, simpler explanations of fundamental concepts like consciousness, life, and gravity. He mentions recent advancements in fusion research, specifically how they have managed to hold plasma in specific shapes for extended periods, which is a significant step forward in energy production. Additionally, he touches on the challenges of modeling quantum mechanical behavior of electrons and the potential role of AI in this area. The conversation emphasizes the importance of exploring new ideas and solutions in physics and related fields.'